### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Función union y unionByName

In [0]:
from pyspark.sql.functions import *

empleados = [(100,'Alfonso','1999','100','H',20000),
             (200,'Maria','2002','200','M',40000),
             (300,'Javiera','2010','100','M',15000),
             (400,'Tomas','2012','300','H',30000)
             ]

columnas = ['id','nombre','año_ingreso','depto_id','genero','salario']

df = spark.createDataFrame(empleados, schema=columnas)

df.printSchema()
df.show()

root
 |-- id: long (nullable = true)
 |-- nombre: string (nullable = true)
 |-- año_ingreso: string (nullable = true)
 |-- depto_id: string (nullable = true)
 |-- genero: string (nullable = true)
 |-- salario: long (nullable = true)

+---+-------+-----------+--------+------+-------+
| id| nombre|año_ingreso|depto_id|genero|salario|
+---+-------+-----------+--------+------+-------+
|100|Alfonso|       1999|     100|     H|  20000|
|200|  Maria|       2002|     200|     M|  40000|
|300|Javiera|       2010|     100|     M|  15000|
|400|  Tomas|       2012|     300|     H|  30000|
+---+-------+-----------+--------+------+-------+



In [0]:
empleados = [(400,'Tomas','2012','300','H',30000),
             (500,'Juan','2002','200','H',40000),
             (600,'Camila','2010','100','M',15000),
             (700,'Sofia','2012','300','M',30000)
             ]

columnas = ['id','nombre','año_ingreso','depto_id','genero','salario']

df_2 = spark.createDataFrame(empleados, schema=columnas)

df_2.printSchema()
df_2.show()

root
 |-- id: long (nullable = true)
 |-- nombre: string (nullable = true)
 |-- año_ingreso: string (nullable = true)
 |-- depto_id: string (nullable = true)
 |-- genero: string (nullable = true)
 |-- salario: long (nullable = true)

+---+------+-----------+--------+------+-------+
| id|nombre|año_ingreso|depto_id|genero|salario|
+---+------+-----------+--------+------+-------+
|400| Tomas|       2012|     300|     H|  30000|
|500|  Juan|       2002|     200|     H|  40000|
|600|Camila|       2010|     100|     M|  15000|
|700| Sofia|       2012|     300|     M|  30000|
+---+------+-----------+--------+------+-------+



#### union

La función PySpark union() se utiliza para combinar dos o más data frames que tienen la misma estructura o esquema. Esta función devuelve un error si el esquema de los data frames difiere entre sí.

Se recomienda utilizar Union por sobre UnionAll, dado que este último ya está en desuso. Con los dos obtengo el mismo resultado.

In [0]:
df.union(df_2).show()

+---+-------+-----------+--------+------+-------+
| id| nombre|año_ingreso|depto_id|genero|salario|
+---+-------+-----------+--------+------+-------+
|100|Alfonso|       1999|     100|     H|  20000|
|200|  Maria|       2002|     200|     M|  40000|
|300|Javiera|       2010|     100|     M|  15000|
|400|  Tomas|       2012|     300|     H|  30000|
|400|  Tomas|       2012|     300|     H|  30000|
|500|   Juan|       2002|     200|     H|  40000|
|600| Camila|       2010|     100|     M|  15000|
|700|  Sofia|       2012|     300|     M|  30000|
+---+-------+-----------+--------+------+-------+



##### Eliminar registros duplicados

**distinct( )** y **dropDuplicates( )** cumplen la misma función.

In [0]:
df.union(df_2).dropDuplicates().show()

+---+-------+-----------+--------+------+-------+
| id| nombre|año_ingreso|depto_id|genero|salario|
+---+-------+-----------+--------+------+-------+
|100|Alfonso|       1999|     100|     H|  20000|
|200|  Maria|       2002|     200|     M|  40000|
|300|Javiera|       2010|     100|     M|  15000|
|400|  Tomas|       2012|     300|     H|  30000|
|500|   Juan|       2002|     200|     H|  40000|
|600| Camila|       2010|     100|     M|  15000|
|700|  Sofia|       2012|     300|     M|  30000|
+---+-------+-----------+--------+------+-------+



#### unionByName

La función PySpark unionByName() también se utiliza para combinar dos o más data frames, pero podría utilizarse para combinar dataframes que tienen diferentes esquemas. Esto se debe a que combina dataframes por el nombre de la columna y no por el orden de las columnas.

##### Ejemplo 1

En este ejemplo, ambos data frames, data_frame1 y data_frame2 tienen el mismo esquema.

In [0]:
data_frame1 = spark.createDataFrame(
	[("Bhuwanesh", 82.98), ("Harshit", 80.31)],
	["Student Name", "Overall Percentage"]
)

data_frame2 = spark.createDataFrame(
	[("Naveen", 91.123), ("Piyush", 90.51)],
	["Student Name", "Overall Percentage"]
)

answer = data_frame1.unionByName(data_frame2)

answer.show()

+------------+------------------+
|Student Name|Overall Percentage|
+------------+------------------+
|   Bhuwanesh|             82.98|
|     Harshit|             80.31|
|      Naveen|            91.123|
|      Piyush|             90.51|
+------------+------------------+



##### Ejercicio 2

En este ejemplo, data_frame1 y data_frame2 tienen un esquema diferente pero la salida es la deseada.

In [0]:
data_frame1 = spark.createDataFrame(
	[("Bhuwanesh", 82.98), ("Harshit", 80.31)],
	["Student Name", "Overall Percentage"]
)

data_frame2 = spark.createDataFrame(
	[(91.123, "Naveen"), (90.51, "Piyush"), (87.67, "Hitesh")],
	["Overall Percentage", "Student Name"]
)

answer = data_frame1.unionByName(data_frame2)

answer.show()

+------------+------------------+
|Student Name|Overall Percentage|
+------------+------------------+
|   Bhuwanesh|             82.98|
|     Harshit|             80.31|
|      Naveen|            91.123|
|      Piyush|             90.51|
|      Hitesh|             87.67|
+------------+------------------+



##### Ejercicio 3

Consideremos ahora dos data frames que contienen un número desigual de columnas (esquema totalmente diferente). En este caso, necesitamos pasar un argumento adicional "allowMissingColumns = True" a la función unionByName.

In [0]:
data_frame1 = spark.createDataFrame(
	[("Bhuwanesh", 82.98, "Computer Science"),
	("Harshit", 80.31, "Information Technology")],
	["Student Name", "Overall Percentage", "Department"]
)

data_frame2 = spark.createDataFrame(
	[("Naveen", 91.123), ("Piyush", 90.51)],
	["Student Name", "Overall Percentage"]
)

res = data_frame1.unionByName(data_frame2, allowMissingColumns=True)

res.show(truncate=False)

+------------+------------------+----------------------+
|Student Name|Overall Percentage|Department            |
+------------+------------------+----------------------+
|Bhuwanesh   |82.98             |Computer Science      |
|Harshit     |80.31             |Information Technology|
|Naveen      |91.123            |null                  |
|Piyush      |90.51             |null                  |
+------------+------------------+----------------------+

